In [6]:
from collections import Counter, OrderedDict
from pathlib import Path
import sys
import textwrap
from tqdm import tqdm
import torch
from typing import Dict, List, Set, Union

from transformers import Seq2SeqTrainingArguments

import unicodedata
import uuid

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl
from aic_nlp_utils.fever import fever_detokenize, import_fever_corpus_from_sqlite
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import stanza
# stanza.download("en")

sys.path.append('Claim_Generation')


%load_ext autoreload
%autoreload 2

This notebook compares various approaches to NER.

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertTokenizerFast
from transformers import pipeline

In [24]:
# CZERT models
# model_name = "/mnt/data/factcheck/models/czert/CZERT-A-ner-CNEC" # BAD
model_name = "/mnt/data/factcheck/models/czert/CZERT-B-ner-CNEC" # BEST 2
# model_name = "/mnt/data/factcheck/models/czert/CZERT-A-ner-BSNLP" # BAD
# model_name = "/mnt/data/factcheck/models/czert/CZERT-B-ner-BSNLP" # OK-
# model_name = "/mnt/data/factcheck/models/czert/PAV-ner-CNEC" # BEST
# model_name = "/mnt/data/factcheck/models/czert/PAV-ner-BSNLP" # OK-
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = BertTokenizerFast(Path(model_name, "vocab.txt"), strip_accents=False, do_lower_case=False)

In [17]:
# Other models
# model_name = "richielo/small-e-czech-finetuned-ner-wikiann" # OK-
model_name = "gamzenurmadan/expanded-multilingual-ner" #OK
# model_name = "davidpc99/multilanguageNER" # OK--
# model_name = "transformersbook/xlm-roberta-base-finetuned-panx-all" # OK--
# model_name = "Andrey1989/mbert-finetuned-ner" # BAD
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [30]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")
# example = "Šéf soukromé žoldnéřské Wagnerovy skupiny Jevgenij Prigožin v úterý obvinil ruskou armádu, že její vojáci uprostřed těžkých bojů prchají ze svých pozic u Bachmutu."

example="Hrozí likvidace malých vinařů, bouří se proti zavedení spotřební daně na víno poslanci napříč politickým spektrem, kteří byli zvoleni v Jihomoravském kraji. Také varují před navýšením byrokracie a ztrátou konkurenceschopnosti. Vládě zavedení spotřební daně na takzvaná tichá vína doporučila Národní ekonomická rada vlády (NERV) jako součást konsolidačního balíčku, který chce premiér Petr Fiala (ODS) představit v polovině května."

# example="Petr Fiala a jeho 5 ministrů Úřadu vlády 14. dubna rozhodlo o rozpočtu na rok 2023."
ner_results = nlp(example)

for r in ner_results:
    print(r)

{'entity_group': 'G', 'score': 0.99717367, 'word': 'Jihomoravském', 'start': 136, 'end': 149}
{'entity_group': 'I', 'score': 0.99903405, 'word': 'Národní ekonomická rada vlády', 'start': 291, 'end': 320}
{'entity_group': 'I', 'score': 0.99836475, 'word': 'NERV', 'start': 322, 'end': 326}


In [28]:
# Nametag, misses many NERs

from ufal.nametag import Ner, Forms, TokenRanges, NamedEntities

class UFALNERExtractor:

    def __init__(self, model):
        self.ner = Ner.load(model)
        self.forms = Forms()
        self.tokens = TokenRanges()
        self.entities = NamedEntities()
        self.tokenizer = self.ner.newTokenizer()
        
    def extract(self, claim):
        self.tokenizer.setText(claim)
        ners = []
        nertypes = []
        while self.tokenizer.nextSentence(self.forms, self.tokens):
            self.ner.recognize(self.forms, self.entities)
            
            entities = sorted(self.entities, key=lambda entity: (entity.start, -entity.length))
            
            prev_end = -1
            for entity in entities:
                if (entity.start + entity.length) <= prev_end: # take only the highest level entities
                    continue
                ners.append(" ".join(self.forms[entity.start:entity.start+entity.length]))
                nertypes.append(entity.type)
                prev_end = entity.start + entity.length

        return ners, nertypes


extractor = UFALNERExtractor("/mnt/data/factcheck/ufal/ner/czech-cnec2.0-140304-no_numbers.ner")

In [31]:
extractor.extract(example)

(['Petr Fiala', 'ODS'], ['P', 'io'])